In [5]:
import altair as alt
import pandas as pd
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, 'states')
living_wage_df =  pd.read_csv('all_living_wage.csv')
living_wage_df.rename(columns={'Work1_Children_0': '1 Working Adult, 0 Child','Work1_Children_1': '1 Working Adult, 1 Child',
                              'Work1_Children_2': '1 Working Adult, 2 Childs','Work2_Children_0': '2 Working Adults, 0 Child',
                                'Work2_Children_1': '2 Working Adults, 1 Child','Work2_Children_2': '2 Working Adults, 2 Childs',
                              'Work1Adult1_Children_0':'1 Working Adult, 1 Nonworking Adult, 0 Child', 
                               'Work1Adult1_Children_1':'1 Working Adult, 1 Nonworking Adult, 1 Child',
                              'Work1Adult1_Children_2':'1 Working Adult, 1 Nonworking Adult, 2 Child'}, inplace=True)

living_wage = pd.melt(living_wage_df, ['State', 'id'])
living_wage.rename(columns={'value': 'Necessary Wage Rate $'}, inplace=True)

dropdown = alt.binding_select(options=list(living_wage['variable'].unique()))
selection = alt.selection_single(name="Family Size",fields=['variable'], bind=dropdown)

choropleth2 = (alt.Chart(living_wage, title="Living Wage").mark_geoshape(
    #   state borders are stroke and strokeWidth
    stroke='black',
    strokeWidth=1
    ).encode(
    #    more color schemes can be found at https://vega.github.io/vega/docs/schemes/
    color=alt.Color('Necessary Wage Rate $', scale=alt.Scale(scheme='bluegreen')),
    tooltip=[alt.Tooltip('State'), alt.Tooltip('Necessary Wage Rate $', format='$')]
    ).add_selection(
        selection
    ).transform_filter(
        selection
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(states, key='id', fields=['type', 'properties', 'geometry'])
).project('albersUsa'
).properties(
        width=800,
        height=600
    )
)

choropleth2.save('Living_Wage.html')

choropleth2

alt.Chart(...)